In [1]:
import numpy as np
from scipy import optimize
import pandas as pd
from collections import namedtuple
import os
import shutil

In [10]:
def read_test_results(fileName):
    """
    ToDO
    """
    tests_dict = {}
    excel = pd.ExcelFile(fileName)
    for sheet in excel.sheet_names:
        df = excel.parse(sheet)
        value = df.values
        print(sheet)
        print(value.shape)
        tests_dict[sheet] = value
    return tests_dict
    


In [9]:
read_test_results('Z.xlsx')

Y
(22817, 2)
Z
(414, 2)


In [14]:
def make_curve(A, n):
    """
    ToDO
    """
    lt_zero = np.arange(-1, 0, 0.1)
    lt_zero_stress = -0.2 * lt_zero
    gt_zero = np.arange(0, 1, 0.01)
    gt_zero_stress = A * np.power(gt_zero, n)
    x = np.hstack([lt_zero, gt_zero])
    y = np.hstack([lt_zero_stress, gt_zero_stress])
    return np.hstack([x.reshape(-1,1), y.reshape(-1,1)])

In [24]:
def write_curve(fileName, curve, lcid):
    """
    ToDO
    """
    with open(fileName, 'w+') as f:
        f.write("*KEY_WORD\n")
        f.write("*DEFINE_CURVE\n")
        f.write(str(int(lcid))+"\n")
        f.write("$#\n")
        
        for p in curve:
            f.write("{:.4f}, {:.4f}\n".format(p[0], p[1]))

        f.write("*END")


In [ ]:
def do_calculate(folder_name, copy_files):
    os.mkdir(folder_name)
    for f in copy_files:
        shutil.copy(folder_name, f)
    os.chdir(folder_name)
    os.c

In [25]:
curve = make_curve(200, 2)
write_curve('lcbb.k', curve, 5000)